In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Getting familiar with the data


In [ ]:
houses = pd.read_csv('/kaggle/input/lianjia/new.csv',  encoding= 'unicode_escape', low_memory=False)
houses.head()



In [ ]:
houses.info()

Got info about every column

I noticed that for the statistics it's better to represent constructionTime as age 

In [ ]:
houses['constructionTime'] = houses['constructionTime'].astype('str') 
condition = (houses.constructionTime == '0') | (houses.constructionTime == 'Î´Öª')
column_name = 'constructionTime'
houses.loc[condition, column_name] = '2018'

condition = (houses.constructionTime == '1')
column_name = 'constructionTime'
houses.loc[condition, column_name] = '2017'
houses['year'] = pd.to_datetime(houses['constructionTime'])
houses['houseAge'] = (pd.datetime.today() - houses['year']).dt.days*0.00273973

In [ ]:
houses.describe()

Got statistical information about every column and added some new, because the dataset was a encoded differentely


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
houses.hist(bins=50, figsize=(20,15))
# save_fig("attribute_histogram_plots")
plt.show()

Got graphical representation of all the numerical values in the dataset.

From here I can see that I have some tail-heavy histograms that I will need to handle somehow to make the better predictions

# Making a dataset

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(houses, test_size=0.2, random_state=42)

# Visualizing the data 


In [ ]:
houses = train_set.copy()
houses.plot(kind="scatter", x="Lng", y="Lat", alpha=0.3,
    s=houses["communityAverage"]/10000, label="Community average", figsize=(10,7),
    c="price", cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()

Checking out if there is any correlations

In [ ]:
corr_matrix = houses.corr()

In [ ]:
corr_matrix["price"].sort_values(ascending=False)

From here we can see that community average is highly correlated with price. Also total price and such parameters as condition of the building, subway availability have some correlation to price. On the opposite the square space of the house has negative correlation which means that the more price per square meter the less square space is there.

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["price", "communityAverage", "DOM",
              "renovationCondition"]
scatter_matrix(houses[attributes], figsize=(12, 8))

To get more correlations I used pandas plotting to see the graphs based on the columns that are the most corralated to price. I got interesting results. For example it seems to be strong correlation between the time the house is on the market(DOM) and its price - the more time it's on market the less it costs. Also the statement the more people(communityAverage) the higher the price per square meter is confirmed on these graphs.

# Preparing data for the algorithm

In [ ]:
from sklearn.impute import SimpleImputer

houses_labels = train_set["price"].copy()
imputer = SimpleImputer(strategy="median")
houses_num = houses.drop(["url", 'id', 'tradeTime', 'livingRoom', 'drawingRoom', 'bathRoom', 'floor', 'constructionTime', 'year'], axis=1)
imputer.fit(houses_num)
test = imputer.transform(houses_num)
houses_trained = pd.DataFrame(test, columns=houses_num.columns,
                          index=houses_num.index)
houses_trained

Here I "cleaned" the dataset numerical values using imputer that replaces them with the average value

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
houses_categories = houses[["floor"]]
housing_cat_encoded = ordinal_encoder.fit_transform(houses_categories)
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(houses_categories)
housing_cat_1hot
housing_cat_1hot.toarray()


From here I noticed that the non-numerical categories are to hard to be transformed, so I just dropped them down

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

price_ix, communityAverage_ix = 6, 18

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_pricePerCommunityAverage=True): # no *args or **kargs
        self.add_pricePerCommunityAverage = add_pricePerCommunityAverage
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        if (X[:, communityAverage_ix] == 0).any():
            pricePerCommunityAverage = X[:, price_ix] / 0.001
        else:
            pricePerCommunityAverage = X[:, price_ix] / X[:, communityAverage_ix]
        if self.add_pricePerCommunityAverage:
            return np.c_[X, pricePerCommunityAverage]

attr_adder = CombinedAttributesAdder(add_pricePerCommunityAverage=False)
houses_extra_attribs = attr_adder.transform(houses.values)

Here I added a function for adding a new extra atribute "pricePerCommunityAverage"

# Adding pipelines


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

houses_num_tr = num_pipeline.fit_transform(houses_num)

Adding pipeline for numerical values

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(houses_num)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
    ])

houses_prepared = full_pipeline.fit_transform(houses)

I have no non-numerical values so I just followed the book

# Selecting and training the model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(houses_prepared, houses_labels)

Train the Linear Regression Model with the prepared data

In [ ]:
some_data = houses.iloc[:10]
some_labels = houses_labels.iloc[:10]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))


See the predictions

In [ ]:
print("Labels:", list(some_labels))

Prediction are really close to the labels, which either mean that model is overfitting the data or it basically is trained well

In [ ]:
from sklearn.metrics import mean_squared_error
houses_predictions = lin_reg.predict(houses_prepared)
lin_mse = mean_squared_error(houses_labels, houses_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Trying to find out the error root-square-mean deviation. The deviation is pretty low 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(houses_prepared, houses_labels)
housing_predictions = tree_reg.predict(houses_prepared)
tree_mse = mean_squared_error(houses_labels, houses_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse


Here I trained a different model(DecisionTreeRegressor) to see what differences there will be with the previous one. This is a powerful model, capable of finding complex nonlinear relationships in the data

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, houses_prepared, houses_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, houses_prepared, houses_labels,
                              scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Here I applied "cross validation" for DecisionTreeRegressor and LinearRegression. Linear regressin showed better scores. Cross validation means that the dataset is shuffled and splitted into K groups. For each of them, the left groups are taken as a training data set and then fit and evaluated on the test set. The scores that were computed after passing all K number of the test shows the quality of the algorithm

# Fine tuning the model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [10, 20], 'max_features': [1, 2]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=2,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(houses_prepared, houses_labels)
grid_search.best_params_

# Evaluating the system on the test set

In [ ]:
final_model = lin_reg

X_test = test_set
y_test = test_set["price"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse